In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sarpyx.processor.core.focus import CoarseRDA
from sarpyx.utils.zarr_utils import ZarrManager 

In [ ]:
file_path = '/Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1a-s1-raw-s-hh-20150519t121652-20150519t121725-005990-007b7d_burst_0.zarr'
handler = ZarrManager(file_path=file_path)
slice = handler.get_slice_block(4)
slice['echo'].shape, slice['echo'].dtype

In [ ]:
focuser = CoarseRDA(raw_data=slice, verbose=True, backend='numpy')

In [5]:
focuser.data_focus()

First FFT along range dimension completed, shape: (15000, 25672)
RAM memory usage: 26.0%
Performing FFT along azimuth dimension (axis=0) with fftshift...
Second FFT along azimuth dimension completed, shape: (15000, 25672)
RAM memory usage: 26.0%
FFT output data shape: (15000, 25672)
- FFT performed successfully!
RAM memory usage: 26.0%
Memory usage: 23.0% -> 26.0% (Δ+3.0%)
Starting range compression...
Input radar data shape: (15000, 25672)
RAM memory usage: 26.0%
Processing with original_w=25672
Starting memory-efficient range compression...
Radar data shape: (15000, 25672)
RAM memory usage: 26.0%
Creating range filter for radar data shape: (15000, 25672)
Range dimension: 25672
TX replica length: 4549
Window type: hamming
Placing replica at indices [10561:15110] in filter of length 25672
Calculated energy for normalization: 0.169079
Range filter shape: (25672,)
Range filter energy after normalization: 3897880053.838840
Second FFT along azimuth dimension completed, shape: (15000, 25672

In [6]:
import numpy as np
import tifffile
from pathlib import Path

def save_radar_magnitude(focuser_obj: 'CoarseRDA', 
                        output_dir: str = '/Data_large/marine/PythonProjects/SAR/sarpyx/data/focused_prev',
                        filename: str = 'radar_magnitude.tiff') -> None:
    """
    Save normalized magnitude of radar data as TIFF.
    
    Args:
        focuser_obj: CoarseRDA object containing radar data
        output_dir: Directory to save the image
        filename: Name of the output file
    """
    assert hasattr(focuser_obj, 'radar_data'), 'Focuser object must have radar_data attribute'
    
    # Create output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Get magnitude and normalize
    magnitude = np.abs(focuser_obj.radar_data)
    vmin, vmax = np.percentile(magnitude, [5, 95])
    
    # Normalize to 0-1 range and convert to float32
    normalized = np.clip((magnitude - vmin) / (vmax - vmin), 0, 1).astype(np.float32)
    
    # Save as TIFF
    full_path = output_path / filename
    tifffile.imwrite(full_path, normalized)
    
    print(f'Saved normalized magnitude to: {full_path}')

# Save the range compressed data
save_radar_magnitude(focuser, filename='az_c_magnitude.tiff')

Saved normalized magnitude to: /Data_large/marine/PythonProjects/SAR/sarpyx/data/focused_prev/az_c_magnitude.tiff


# Single Steps:

In [7]:
# focuser.range_compression('hamming')